<a href="https://colab.research.google.com/github/SofiaMarchesi/House-Prices_k/blob/master/House_Prices_data_analysis%5B5%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive',force_remount=True)
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/houses_train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/houses_test.csv')

Mounted at /content/drive


# 1.   Libraries



In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import statsmodels as sm

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge,Lasso,ElasticNet,SGDRegressor,Perceptron
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor,VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error,mean_squared_error


from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_validate
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
import sklearn
sklearn.__version__

'0.22.2.post1'

In [5]:
pip install dmba

In [6]:
import dmba

no display found. Using non-interactive Agg backend


# 2.   Copy raw data



In [7]:
train1 = train.copy()
test1 = test.copy()
print(train1.shape)
print(test1.shape)


(1460, 81)
(1459, 80)


In [8]:
pd.set_option('display.max_rows', train1.shape[0]+1)

In [9]:
print(display(train1.head(10)))



,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,...,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,...,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,...,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,...,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,

None


In [10]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [11]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [12]:
train1.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


# 3. Treating NaN values
- Make a function to fill null (NaN) values with a given value
- In categorical/object, it will be fill with 'NA' or 'None', depends of the type of the feature

In [13]:
def FillNa(df,value):
  df = df.fillna(value)
  print(df.value_counts(sort=True).iloc[:11])
  return df

# 3.1. Alley
- Analyse 'Alley' - lot of values null: 1369 - 93%
- There is 3 types: Gravel, Paved and No alley acess.
- NaN = No Alley Acess, fill with 'NA'

In [14]:
train1[~train1.Alley.isna()].groupby('Alley')['Alley'].count()

Alley
Grvl    50
Pave    41
Name: Alley, dtype: int64

In [15]:
train1['Alley']=FillNa(train1.Alley,'NA')
test1['Alley']=FillNa(test1.Alley,'NA')

NA      1369
Grvl      50
Pave      41
Name: Alley, dtype: int64
NA      1352
Grvl      70
Pave      37
Name: Alley, dtype: int64


# 3.2. MasVnrType and MasVnrArea
- Analyse MasVnrType e MasVnrArea, both with the same number of NaN: 8
- The NaN refers to the same houses
- [MasVnrType] Great number of houses is in 'None' (59%), so, will be fill with 'None' the 'NaN'.
- [MasVnrArea] with the same logic of MasVnrType, but fill with 0.0, because in this case, is a continuous value.

In [16]:
#The same houses have the 'NaN' in the both features, so the treat will be the same
print(train1['MasVnrType'][train1.MasVnrType.isna()])
print(train1['MasVnrArea'][train1.MasVnrArea.isna()])

234     NaN
529     NaN
650     NaN
936     NaN
973     NaN
977     NaN
1243    NaN
1278    NaN
Name: MasVnrType, dtype: object
234    NaN
529    NaN
650    NaN
936    NaN
973    NaN
977    NaN
1243   NaN
1278   NaN
Name: MasVnrArea, dtype: float64


In [17]:
train1.MasVnrType.value_counts(normalize=True)

None       0.595041
BrkFace    0.306474
Stone      0.088154
BrkCmn     0.010331
Name: MasVnrType, dtype: float64

In [18]:
train1.MasVnrArea.value_counts(normalize=True,sort=True).iloc[:11]

0.0      0.592975
72.0     0.005510
180.0    0.005510
108.0    0.005510
120.0    0.004821
16.0     0.004821
80.0     0.004132
200.0    0.004132
106.0    0.004132
340.0    0.004132
170.0    0.003444
Name: MasVnrArea, dtype: float64

In [19]:
train1['MasVnrType'] = FillNa(train1['MasVnrType'],'None')
train1['MasVnrArea'] = FillNa(train1['MasVnrArea'],0.0)

test1['MasVnrType']=FillNa(test1['MasVnrType'],'None')
test1['MasVnrArea'] = FillNa(test1['MasVnrArea'],0.0)


None       872
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64
0.0      869
72.0       8
180.0      8
108.0      8
120.0      7
16.0       7
80.0       6
200.0      6
106.0      6
340.0      6
170.0      5
Name: MasVnrArea, dtype: int64
None       894
BrkFace    434
Stone      121
BrkCmn      10
Name: MasVnrType, dtype: int64
0.0      892
176.0     10
144.0      9
120.0      8
216.0      8
200.0      7
128.0      6
504.0      6
302.0      6
198.0      6
196.0      5
Name: MasVnrArea, dtype: int64


In [20]:
train1.MasVnrType.shape

(1460,)

In [21]:
train1.MasVnrArea.shape

(1460,)

# 3.3. BsmtQual, BsmtCond, BstmExposure, BstmFinType1, BstmFinType2
- All with 37 NaN, and refers to the same houses.
- Will be fill with 'NA'

In [22]:
index_Qual = train1.BsmtQual[train1.BsmtQual.isna()].index
index_Cond = train1.BsmtCond[train1.BsmtCond.isna()].index
index_Exp = train1.BsmtExposure[train1.BsmtExposure.isna()].index
index_Type1 = train1.BsmtFinType1[train1.BsmtFinType1.isna()].index
index_Type2 = train1.BsmtFinType2[train1.BsmtFinType2.isna()].index

In [23]:
print(train1.BsmtQual.value_counts())
print(train1.BsmtCond.value_counts())
print(train1.BsmtExposure.value_counts())
print(train1.BsmtFinType1.value_counts())
print(train1.BsmtFinType2.value_counts())

TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64
TA    1311
Gd      65
Fa      45
Po       2
Name: BsmtCond, dtype: int64
No    953
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
Name: BsmtFinType1, dtype: int64
Unf    1256
Rec      54
LwQ      46
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64


In [24]:
train1.BsmtQual = FillNa(train1.BsmtQual,'NA')
train1.BsmtCond = FillNa(train1.BsmtCond,'NA')
train1.BsmtExposure = FillNa(train1.BsmtExposure,'NA')
train1.BsmtFinType1 = FillNa(train1.BsmtFinType1,'NA')
train1.BsmtFinType2 = FillNa(train1.BsmtFinType2,'NA')


test1.BsmtQual = FillNa(test1.BsmtQual,'NA')
test1.BsmtCond = FillNa(test1.BsmtCond,'NA')
test1.BsmtExposure = FillNa(test1.BsmtExposure,'NA')
test1.BsmtFinType1 = FillNa(test1.BsmtFinType1,'NA')
test1.BsmtFinType2 = FillNa(test1.BsmtFinType2,'NA')


TA    649
Gd    618
Ex    121
NA     37
Fa     35
Name: BsmtQual, dtype: int64
TA    1311
Gd      65
Fa      45
NA      37
Po       2
Name: BsmtCond, dtype: int64
No    953
Av    221
Gd    134
Mn    114
NA     38
Name: BsmtExposure, dtype: int64
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
NA      37
Name: BsmtFinType1, dtype: int64
Unf    1256
Rec      54
LwQ      46
NA       38
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64
TA    634
Gd    591
Ex    137
Fa     53
NA     44
Name: BsmtQual, dtype: int64
TA    1295
Fa      59
Gd      57
NA      45
Po       3
Name: BsmtCond, dtype: int64
No    951
Av    197
Gd    142
Mn    125
NA     44
Name: BsmtExposure, dtype: int64
GLQ    431
Unf    421
ALQ    209
Rec    155
BLQ    121
LwQ     80
NA      42
Name: BsmtFinType1, dtype: int64
Unf    1237
Rec      51
NA       42
LwQ      41
BLQ      35
ALQ      33
GLQ      20
Name: BsmtFinType2, dtype: int64


# 3.4. Electrical 
- 1 NaN
- Fill with the most commmon: 'SBrkr'

In [25]:
train1.Electrical.value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [26]:
train1.Electrical = FillNa(train1.Electrical,'SBrkr')

test1.Electrical = FillNa(test1.Electrical,'SBrkr')

SBrkr    1335
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64
SBrkr    1337
FuseA      94
FuseF      23
FuseP       5
Name: Electrical, dtype: int64


# 3.5. FirePlacesQu
- 690 NaN
- Verifying the values of 'FirePlace', witch are complete, there are 690 houses without fireplaces and 690 'NaN' in 'FirePlaceQu'. So, it will be fill with 'NA'.


In [27]:
print(train1.Fireplaces.value_counts())

0    690
1    650
2    115
3      5
Name: Fireplaces, dtype: int64


In [28]:
train1.FireplaceQu = FillNa(train1.FireplaceQu,'NA')
test1.FireplaceQu = FillNa(test1.FireplaceQu,'NA')

NA    690
Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64
NA    730
Gd    364
TA    279
Fa     41
Po     26
Ex     19
Name: FireplaceQu, dtype: int64


# 3.6.[Garages] GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageAreas,GarageQual, GarageCond
- Have 81 NaN.
- Compare the GarageCars and GarageArea witch are complete, with the incomplete features.
- So, the number of 0 in Garagecars match with the NaN of the another NaN features. Will be fill with 'NA'

In [29]:
print(train1.GarageCars.value_counts())
print(train1.GarageType.value_counts())


train1.GarageType=FillNa(train1.GarageType,'NA')
train1.GarageYrBlt=FillNa(train1.GarageYrBlt,0)
train1.GarageFinish=FillNa(train1.GarageFinish,'NA')
train1.GarageQual=FillNa(train1.GarageQual,'NA')
train1.GarageCond=FillNa(train1.GarageCond,'NA')


test1.GarageType=FillNa(test1.GarageType,'NA')
test1.GarageYrBlt=FillNa(test1.GarageYrBlt,0.0)
test1.GarageFinish=FillNa(test1.GarageFinish,'NA')
test1.GarageQual=FillNa(test1.GarageQual,'NA')
test1.GarageCond=FillNa(test1.GarageCond,'NA')


2    824
1    369
3    181
0     81
4      5
Name: GarageCars, dtype: int64
Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64
Attchd     870
Detchd     387
BuiltIn     88
NA          81
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64
0.0       81
2005.0    65
2006.0    59
2004.0    53
2003.0    50
2007.0    49
1977.0    35
1998.0    31
1999.0    30
1976.0    29
2008.0    29
Name: GarageYrBlt, dtype: int64
Unf    605
RFn    422
Fin    352
NA      81
Name: GarageFinish, dtype: int64
TA    1311
NA      81
Fa      48
Gd      14
Ex       3
Po       3
Name: GarageQual, dtype: int64
TA    1326
NA      81
Fa      35
Gd       9
Po       7
Ex       2
Name: GarageCond, dtype: int64
Attchd     853
Detchd     392
BuiltIn     98
NA          76
Basment     17
2Types      17
CarPort      6
Name: GarageType, dtype: int64
0.0       78
2005.0    77
2007.0    66
2006.0    56
2004.0    46
2003.0    42
2008.0 

# 3.7. Pool
- PoolArea,PoolQC(7 Non-null)
- Just 7 with values! Probably, this feature will be drop. But now, will fill with 'NA'

In [30]:
train1.PoolQC[~train1.PoolQC.isna()]


197     Ex
810     Fa
1170    Gd
1182    Ex
1298    Gd
1386    Fa
1423    Gd
Name: PoolQC, dtype: object

In [31]:
train1.PoolArea.value_counts()

0      1453
738       1
648       1
576       1
555       1
519       1
512       1
480       1
Name: PoolArea, dtype: int64

In [32]:
train1.PoolQC=FillNa(train1.PoolQC,'NA')
test1.PoolQC=FillNa(test1.PoolQC,'NA')

NA    1453
Gd       3
Ex       2
Fa       2
Name: PoolQC, dtype: int64
NA    1456
Ex       2
Gd       1
Name: PoolQC, dtype: int64


# 3.8. Fence 
- 281 non-null
- Fill NaN with 'NA'

In [33]:
train1.Fence.value_counts()

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

In [34]:
train1.Fence = FillNa(train1.Fence,'NA')
test1.Fence = FillNa(test1.Fence,'NA')

NA       1179
MnPrv     157
GdPrv      59
GdWo       54
MnWw       11
Name: Fence, dtype: int64
NA       1169
MnPrv     172
GdPrv      59
GdWo       58
MnWw        1
Name: Fence, dtype: int64


# 3.9. MiscFeature
- 54 Non-null
- **Don't have 'Elev' in the feature.**
- Fill with NA in this first analysis

In [35]:
train1.MiscFeature.value_counts()

Shed    49
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

In [36]:
train1.MiscFeature = FillNa(train1.MiscFeature,'NA')

test1.MiscFeature = FillNa(test1.MiscFeature,'NA')

NA      1406
Shed      49
Gar2       2
Othr       2
TenC       1
Name: MiscFeature, dtype: int64
NA      1408
Shed      46
Gar2       3
Othr       2
Name: MiscFeature, dtype: int64


# 4. Transform some features in to categoricals:
1. Just change the dtype
or
2. Change to categorical with numerical scale.

# 4.1. Copy2 = train2 and test2

In [37]:
train2 = train1.copy()
test2 = test1.copy()

In [38]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [39]:
categorical = ['MSSubClass','MSZoning','Street','Alley','LotShape','LandContour', 'Utilities', 'LotConfig','LandSlope', 'Neighborhood', 
               'Condition1', 'Condition2','BldgType','HouseStyle','RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType','Foundation',
               'Heating','HeatingQC','Electrical','GarageType','MiscFeature', 'SaleType','SaleCondition','OverallQual','OverallCond','ExterQual',
               'ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','KitchenQual','Functional','Fireplaces','FireplaceQu',
               'GarageYrBlt','GarageFinish','GarageCars','GarageQual','GarageCond','PoolQC','Fence','MoSold','YrSold','CentralAir','PavedDrive']
len(categorical)

51

In [40]:
train2[categorical] = train2[categorical].astype('category')
test2[categorical] = test2[categorical].astype('category')

In [41]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1460 non-null   int64   
 1   MSSubClass     1460 non-null   category
 2   MSZoning       1460 non-null   category
 3   LotFrontage    1201 non-null   float64 
 4   LotArea        1460 non-null   int64   
 5   Street         1460 non-null   category
 6   Alley          1460 non-null   category
 7   LotShape       1460 non-null   category
 8   LandContour    1460 non-null   category
 9   Utilities      1460 non-null   category
 10  LotConfig      1460 non-null   category
 11  LandSlope      1460 non-null   category
 12  Neighborhood   1460 non-null   category
 13  Condition1     1460 non-null   category
 14  Condition2     1460 non-null   category
 15  BldgType       1460 non-null   category
 16  HouseStyle     1460 non-null   category
 17  OverallQual    1460 non-null   ca

In [42]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1459 non-null   int64   
 1   MSSubClass     1459 non-null   category
 2   MSZoning       1455 non-null   category
 3   LotFrontage    1232 non-null   float64 
 4   LotArea        1459 non-null   int64   
 5   Street         1459 non-null   category
 6   Alley          1459 non-null   category
 7   LotShape       1459 non-null   category
 8   LandContour    1459 non-null   category
 9   Utilities      1457 non-null   category
 10  LotConfig      1459 non-null   category
 11  LandSlope      1459 non-null   category
 12  Neighborhood   1459 non-null   category
 13  Condition1     1459 non-null   category
 14  Condition2     1459 non-null   category
 15  BldgType       1459 non-null   category
 16  HouseStyle     1459 non-null   category
 17  OverallQual    1459 non-null   ca

# 4.2. Change some features to numerical scale

In [43]:
train2['MSSubClass'].unique()

[60, 20, 70, 50, 190, ..., 80, 160, 75, 180, 40]
Length: 15
Categories (15, int64): [60, 20, 70, 50, ..., 160, 75, 180, 40]

In [44]:
train2['MSSubClass']=train2['MSSubClass'].cat.rename_categories({20:'A',30:'B',40:'C',45:'D',50:'E',
                                                                 60:'F',70:'G',75:'H',80:'I',85:'J',
                                                                 90:'L',120:'M',150:'N',160:'O',180:'P'
                                                                 ,190:'Q'})

test2['MSSubClass']=test2['MSSubClass'].cat.rename_categories({20:'A',30:'B',40:'C',45:'D',50:'E',
                                                                 60:'F',70:'G',75:'H',80:'I',85:'J',
                                                                 90:'L',120:'M',150:'N',160:'O',180:'P'
                                                                 ,190:'Q'})


In [45]:
print(train2.MSSubClass.value_counts())

print(test2.MSSubClass.value_counts())



A    536
F    299
E    144
M     87
B     69
O     63
G     60
I     58
L     52
Q     30
J     20
H     16
D     12
P     10
C      4
Name: MSSubClass, dtype: int64
A    543
F    276
E    143
M     95
B     70
G     68
O     65
I     60
L     57
Q     31
J     28
P      7
H      7
D      6
C      2
N      1
Name: MSSubClass, dtype: int64


In [46]:
train2['ExterQual'].unique()

[Gd, TA, Ex, Fa]
Categories (4, object): [Gd, TA, Ex, Fa]

In [47]:
train2['ExterQual']=train2['ExterQual'].cat.rename_categories({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})

test2['ExterQual']=test2['ExterQual'].cat.rename_categories({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})

In [48]:
train2['ExterCond']=train2['ExterCond'].cat.rename_categories(({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1}))
test2['ExterCond']=test2['ExterCond'].cat.rename_categories(({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1}))

In [49]:
train2['BsmtQual']=train2['BsmtQual'].cat.rename_categories({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0})
test2['BsmtQual']=test2['BsmtQual'].cat.rename_categories({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0})

In [50]:
train2['BsmtCond']=train2['BsmtCond'].cat.rename_categories({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0})
test2['BsmtCond']=test2['BsmtCond'].cat.rename_categories({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0})

In [51]:
train2['BsmtExposure']=train2['BsmtExposure'].cat.rename_categories({'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0})

test2['BsmtExposure']=test2['BsmtExposure'].cat.rename_categories({'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0})

In [52]:
train2['BsmtFinType1'].unique()

[GLQ, ALQ, Unf, Rec, BLQ, NA, LwQ]
Categories (7, object): [GLQ, ALQ, Unf, Rec, BLQ, NA, LwQ]

In [53]:
train2['BsmtFinType1']=train2['BsmtFinType1'].cat.rename_categories({'GLQ':6, 'ALQ':5, 'Unf':1, 'Rec':3, 'BLQ':4, 'NA':0, 'LwQ':2})

test2['BsmtFinType1']=test2['BsmtFinType1'].cat.rename_categories({'GLQ':6, 'ALQ':5, 'Unf':1, 'Rec':3, 'BLQ':4, 'NA':0, 'LwQ':2})

In [54]:
train2['BsmtFinType2']=train2['BsmtFinType2'].cat.rename_categories({'GLQ':6, 'ALQ':5, 'Unf':1, 'Rec':3, 'BLQ':4, 'NA':0, 'LwQ':2})

test2['BsmtFinType2']=test2['BsmtFinType2'].cat.rename_categories({'GLQ':6, 'ALQ':5, 'Unf':1, 'Rec':3, 'BLQ':4, 'NA':0, 'LwQ':2})



In [55]:
train2['GarageQual'].unique()

[TA, Fa, Gd, NA, Ex, Po]
Categories (6, object): [TA, Fa, Gd, NA, Ex, Po]

In [56]:
train2['KitchenQual']=train2['KitchenQual'].cat.rename_categories({'Gd':4, 'TA':3, 'Ex':5, 'Fa':2,'Po':1})

test2['KitchenQual']=test2['KitchenQual'].cat.rename_categories({'Gd':4, 'TA':3, 'Ex':5, 'Fa':2,'Po':1})

In [57]:
train2['Functional']=train2['Functional'].cat.rename_categories({'Typ':8, 'Min1':7, 'Maj1':4, 'Min2':6, 'Mod':5, 'Maj2':3, 'Sev':2,'Sal':1})

test2['Functional']=test2['Functional'].cat.rename_categories({'Typ':8, 'Min1':7, 'Maj1':4, 'Min2':6, 'Mod':5, 'Maj2':3, 'Sev':2,'Sal':1})

In [58]:
train2['FireplaceQu']=train2['FireplaceQu'].cat.rename_categories({'NA':0, 'TA':3, 'Gd':4, 'Fa':2, 'Ex':5, 'Po':1})

test2['FireplaceQu']=test2['FireplaceQu'].cat.rename_categories({'NA':0, 'TA':3, 'Gd':4, 'Fa':2, 'Ex':5, 'Po':1})

In [59]:
train2['GarageFinish']=train2['GarageFinish'].cat.rename_categories({'RFn':2, 'Unf':1, 'Fin':3, 'NA':0})

test2['GarageFinish']=test2['GarageFinish'].cat.rename_categories({'RFn':2, 'Unf':1, 'Fin':3, 'NA':0})

In [60]:
train2['GarageQual']=train2['GarageQual'].cat.rename_categories({'TA':3, 'Fa':2, 'Gd':4, 'NA':0, 'Ex':5, 'Po':1})
test2['GarageQual']=test2['GarageQual'].cat.rename_categories({'TA':3, 'Fa':2, 'Gd':4, 'NA':0, 'Ex':5, 'Po':1})

In [61]:
train2['GarageCond']=train2['GarageCond'].cat.rename_categories({'TA':3, 'Fa':2, 'Gd':4, 'NA':0, 'Ex':5, 'Po':1})
test2['GarageCond']=test2['GarageCond'].cat.rename_categories({'TA':3, 'Fa':2, 'Gd':4, 'NA':0, 'Ex':5, 'Po':1})

In [62]:
train2['PavedDrive'].unique()

[Y, N, P]
Categories (3, object): [Y, N, P]

In [63]:
train2['PavedDrive']=train2['PavedDrive'].cat.rename_categories({'Y':2,'P':1,'N':0})

test2['PavedDrive']=test2['PavedDrive'].cat.rename_categories({'Y':2,'P':1,'N':0})


In [64]:
train2['CentralAir'].unique()

[Y, N]
Categories (2, object): [Y, N]

In [65]:
train2['PoolQC']=train2['PoolQC'].cat.rename_categories({'NA':0, 'Ex':4, 'Fa':1, 'Gd':3,'TA':2})

test2['PoolQC']=test2['PoolQC'].cat.rename_categories({'NA':0, 'Ex':4, 'Fa':1, 'Gd':3,'TA':2})

In [66]:
train2['CentralAir']=train2['CentralAir'].cat.rename_categories({'Y':1,'N':0})
test2['CentralAir']=test2['CentralAir'].cat.rename_categories({'Y':1,'N':0})


In [67]:
train2['HeatingQC']=train2['HeatingQC'].cat.rename_categories({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})

test2['HeatingQC']=test2['HeatingQC'].cat.rename_categories({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})

In [68]:
test3 = test2.copy()
train3 = train2.copy()

In [69]:
test3[categorical].describe()

,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Heating,HeatingQC,Electrical,GarageType,MiscFeature,SaleType,SaleCondition,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,KitchenQual,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageQual,GarageCond,PoolQC,Fence,MoSold,YrSold,CentralAir,PavedDrive
count,1459,1455,1459,1459,1459,1459,1457,1459,1459,1459,1459,1459,1459,1459,1459,1459,1458,1458,1459,1459,1459,1459,1459,1459,1459,1458,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459,1458,1457,1459,1459,1459.0,1459,1458.0,1459,1459,1459,1459,1459,1459,1459,1459
unique,16,5,2,3,4,4,1,5,3,25,9,5,5,7,6,4,13,15,4,6,4,5,4,7,4,9,6,10,9,4,5,5,5,5,7,7,4,7,5,6,98.0,4,6.0,5,6,3,5,12,5,2,3
top,A,RL,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,GasA,5,SBrkr,Attchd,NA,WD,Normal,5,5,3,3,3,3,1,6,1,3,8,0,0,0.0,1,2.0,3,3,0,NA,6,2007,1,2
freq,543,1114,1453,1352,934,1311,1457,1081,1396,218,1251,1444,1205,745,1169,1442,510,510,894,661,1446,752,1337,853,1408,1258,1204,428,824,892,1256,634,1295,951,431,1237,757,1357,730,730,78.0,625,770.0,1293,1328,1456,1169,250,363,1358,1301


In [70]:
train3[categorical].describe()

,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Heating,HeatingQC,Electrical,GarageType,MiscFeature,SaleType,SaleCondition,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,KitchenQual,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageQual,GarageCond,PoolQC,Fence,MoSold,YrSold,CentralAir,PavedDrive
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460.0,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460
unique,15,5,2,3,4,4,2,5,3,25,9,8,5,8,6,8,15,16,4,6,6,5,5,7,5,9,6,10,9,4,5,5,5,5,7,7,4,7,4,6,98.0,4,5,6,6,4,5,12,5,2,3
top,A,RL,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,GasA,5,SBrkr,Attchd,NA,WD,Normal,5,5,3,3,3,3,1,1,1,3,8,0,0,0.0,1,2,3,3,0,NA,6,2009,1,2
freq,536,1151,1454,1369,925,1311,1459,1052,1382,225,1260,1445,1220,726,1141,1434,515,504,872,647,1428,741,1335,870,1406,1267,1198,397,821,906,1282,649,1311,953,430,1256,735,1360,690,690,81.0,605,824,1311,1326,1453,1179,253,338,1365,1340


# Fill NaN of test values test3/train3

In [71]:
test3 = test2.copy()
train3 = train2.copy()

In [72]:
# Fill with the most frequently value - category values
test_nan_categ =['MSZoning','Utilities','Exterior1st','Exterior2nd','KitchenQual','Functional','GarageCars','SaleType']

#Fill with de mean - float and integer values
test_nan_float= ['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','GarageArea']

In [73]:
fill_float = test3[test_nan_float].mean().round()
fill_float

BsmtFinSF1       439.0
BsmtFinSF2        53.0
BsmtUnfSF        554.0
TotalBsmtSF     1046.0
BsmtFullBath       0.0
BsmtHalfBath       0.0
GarageArea       473.0
dtype: float64

In [74]:
test3 = test3.fillna(fill_float)

- Fill the category values with most frequently

In [75]:
test3 = test3.fillna({'MSZoning':test3.MSZoning.value_counts().index[0]})
test3 = test3.fillna({'Utilities':test3.Utilities.value_counts().index[0]})
test3 = test3.fillna({'Exterior2nd':test3.Exterior2nd.value_counts().index[0]})
test3 = test3.fillna({'KitchenQual':test3.KitchenQual.value_counts().index[0]})
test3 = test3.fillna({'Functional':test3.Functional.value_counts().index[0]})
test3 = test3.fillna({'GarageCars':test3.GarageCars.value_counts().index[0]})
test3 = test3.fillna({'SaleType':test3.SaleType.value_counts().index[0]})

# 4.3. Features engineering train_eng/test_eng

In [76]:
train_eng = train3.copy()
test_eng = test3.copy()

In [77]:
train_eng['All_Bath'] = train_eng['FullBath'] + 1/2*train_eng['HalfBath']

test_eng['All_Bath'] = test_eng['FullBath'] + 1/2*test_eng['HalfBath']

In [78]:
train_eng['Total_Area'] = train_eng['TotalBsmtSF'] + train_eng['1stFlrSF'] + train_eng['2ndFlrSF'] + train_eng['GrLivArea'] + train_eng['GarageArea']

test_eng['Total_Area'] = test_eng['TotalBsmtSF'] + test_eng['1stFlrSF'] + test_eng['2ndFlrSF'] + test_eng['GrLivArea'] + test_eng['GarageArea']

# Division of NaN and Not NaN - LotFrontage

# 3.10. LotFrontage fill NA

In [79]:
train4 = train_eng.copy()
test4 = test_eng.copy()

In [80]:
train_lotfr = train4.corr().LotFrontage.sort_values(ascending=False)
train_lotfr

LotFrontage      1.000000
Total_Area       0.462841
1stFlrSF         0.457181
LotArea          0.426095
GrLivArea        0.402797
TotalBsmtSF      0.392075
TotRmsAbvGrd     0.352096
SalePrice        0.351799
GarageArea       0.344997
BedroomAbvGr     0.263170
BsmtFinSF1       0.233633
PoolArea         0.206167
FullBath         0.198769
All_Bath         0.194845
MasVnrArea       0.192411
OpenPorchSF      0.151972
BsmtUnfSF        0.132644
YearBuilt        0.123349
BsmtFullBath     0.100949
YearRemodAdd     0.088866
WoodDeckSF       0.088521
2ndFlrSF         0.080177
3SsnPorch        0.070029
HalfBath         0.053532
BsmtFinSF2       0.049900
ScreenPorch      0.041383
LowQualFinSF     0.038469
EnclosedPorch    0.010700
MiscVal          0.003368
KitchenAbvGr    -0.006069
BsmtHalfBath    -0.007234
Id              -0.010601
Name: LotFrontage, dtype: float64

In [81]:
list_lotfr = list(train_lotfr.index.drop(['Id','LotFrontage']))
list_lotfr

['Total_Area',
 '1stFlrSF',
 'LotArea',
 'GrLivArea',
 'TotalBsmtSF',
 'TotRmsAbvGrd',
 'SalePrice',
 'GarageArea',
 'BedroomAbvGr',
 'BsmtFinSF1',
 'PoolArea',
 'FullBath',
 'All_Bath',
 'MasVnrArea',
 'OpenPorchSF',
 'BsmtUnfSF',
 'YearBuilt',
 'BsmtFullBath',
 'YearRemodAdd',
 'WoodDeckSF',
 '2ndFlrSF',
 '3SsnPorch',
 'HalfBath',
 'BsmtFinSF2',
 'ScreenPorch',
 'LowQualFinSF',
 'EnclosedPorch',
 'MiscVal',
 'KitchenAbvGr',
 'BsmtHalfBath']

In [82]:
g = sns.PairGrid(train4,x_vars=list_lotfr[:11],y_vars='LotFrontage')
g.map(sns.scatterplot)
g = sns.PairGrid(train4,x_vars=list_lotfr[11:20],y_vars='LotFrontage')
g.map(sns.scatterplot)

In [83]:
train5 = train4.copy()
test5 = test4.copy()

In [84]:
drop1 =  train5.drop(labels = train5[train5['LotFrontage']>300].index)
drop2 = drop1.drop(labels = drop1[drop1['LotArea']>200000].index)

In [85]:
g = sns.PairGrid(drop2,x_vars=list_lotfr[:11],y_vars='LotFrontage')
g.map(sns.scatterplot)

g2 = sns.PairGrid(drop2,x_vars=list_lotfr[11:20],y_vars='SalePrice')
g2.map(sns.scatterplot)

In [86]:

f2, ax2 = plt.subplots(figsize=(12,12))
sns.regplot(x='1stFlrSF',y='LotFrontage',data=drop2)

f3, ax3 = plt.subplots(figsize=(12,12))
sns.regplot(x='LotArea',y='LotFrontage',data=drop2)

In [87]:
train_lotfr = drop2.corr().LotFrontage.drop(['FullBath','GarageArea','HalfBath','1stFlrSF','2ndFlrSF','GrLivArea','TotalBsmtSF']).sort_values(ascending=False)
train_lotfr_list = list(train_lotfr.index.drop(['Id','LotFrontage','MiscVal','KitchenAbvGr']))
train_lotfr

LotFrontage      1.000000
LotArea          0.487970
Total_Area       0.429230
SalePrice        0.377006
TotRmsAbvGrd     0.342475
BedroomAbvGr     0.275212
All_Bath         0.199097
MasVnrArea       0.185391
BsmtUnfSF        0.161085
BsmtFinSF1       0.157541
OpenPorchSF      0.142570
YearBuilt        0.128740
PoolArea         0.122993
WoodDeckSF       0.097166
YearRemodAdd     0.081539
3SsnPorch        0.079487
BsmtFullBath     0.063763
ScreenPorch      0.051356
BsmtFinSF2       0.045289
LowQualFinSF     0.045150
EnclosedPorch    0.019699
MiscVal          0.006591
KitchenAbvGr    -0.001922
BsmtHalfBath    -0.003002
Id              -0.025777
Name: LotFrontage, dtype: float64

In [88]:
train6 = drop2.copy()
#test4 = drop3_true.copy()
test6 = test5.copy()

In [89]:
# Filtrar os valores com NaN e os sem NaN
#Sem NaN, serão os valores do modelo
train_notnan = train6[~train6.LotFrontage.isna()]
#Com NaN, os que serão preditados
train_nan = train6[train6.LotFrontage.isna()]

In [90]:
# Filtrar os valores com NaN e os sem NaN
#Sem NaN, serão os valores do modelo
test_notnan_true = test6[~test6.LotFrontage.isna()]
#Com NaN, os que serão preditados
test_nan_true= test6[test6.LotFrontage.isna()]

In [91]:
train_nan = train_nan.drop('LotFrontage',axis=1)

In [92]:
test_nan_true = test_nan_true.drop('LotFrontage',axis=1)

In [93]:
train_nan.head(10)

,Id,MSSubClass,MSZoning,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,...,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,All_Bath,Total_Area
7,8,F,RL,10382,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,3,3,CBlock,4,3,2,5,859,4,32,216,1107,GasA,5,...,1107,983,0,2090,1,0,2,1,3,1,3,7,8,2,3,Attchd,1973.0,2,2,484,3,3,2,235,204,228,0,0,0,0,NA,Shed,350,11,2009,WD,Normal,200000,2.5,5771
12,13,A,RL,12968,Pave,NA,IR2,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,6,1962,1962,Hip,CompShg,HdBoard,Plywood,None,0.0,3,3,CBlock,3,3,1,5,737,1,0,175,912,GasA,3,...,912,0,0,912,1,0,1,0,2,1,3,4,8,0,0,Detchd,1962.0,1,1,352,3,3,2,140,0,0,0,176,0,0,NA,NA,0,9,2008,WD,Normal,144000,1.0,3088
14,15,A,RL,10920,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,MetalSd,MetalSd,BrkFace,212.0,3,3,CBlock,3,3,1,4,733,1,0,520,1253,GasA,3,...,1253,0,0,1253,1,0,1,1,2,1,3,5,8,1,2,Attchd,1960.0,2,1,352,3,3,2,0,213,176,0,0,0,0,GdWo,NA,0,5,2008,WD,Normal,157000,1.5,4111
16,17,A,RL,11241,Pave,NA,IR1,Lvl,AllPub,CulDSac,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,7,1970,1970,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,180.0,3,3,CBlock,3,3,1,5,578,1,0,426,1004,GasA,5,...,1004,0,0,1004,1,0,1,0,2,1,3,5,8,1,3,Attchd,1970.0,3,2,480,3,3,2,0,0,0,0,0,0,0,NA,Shed,700,3,2010,WD,Normal,149000,1.0,3492
24,25,A,RL,8246,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,8,1968,2001,Gable,CompShg,Plywood,Plywood,None,0.0,3,4,CBlock,3,3,2,3,188,5,668,204,1060,GasA,5,...,1060,0,0,1060,1,0,1,0,3,1,4,6,8,1,3,Attchd,1968.0,1,1,270,3,3,2,406,90,0,0,0,0,0,MnPrv,NA,0,5,2010,WD,Normal,154000,1.0,3450
31,32,A,RL,8544,Pave,NA,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,6,1966,2006,Gable,CompShg,HdBoard,HdBoard,None,0.0,3,3,CBlock,3,3,1,1,0,1,0,1228,1228,GasA,4,...,1228,0,0,1228,0,0,1,1,3,1,4,6,8,0,0,Attchd,1966.0,1,1,271,3,3,2,0,65,0,0,0,0,0,MnPrv,NA,0,6,2008,WD,Normal,149350,1.5,3955
42,43,J,RL,9180,Pave,NA,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,SFoyer,5,7,1983,1983,Gable,CompShg,HdBoard,HdBoard,None,0.0,3,3,CBlock,4,3,3,5,747,2,93,0,840,GasA,4,...,884,0,0,884,1,0,1,0,2,1,4,5,8,0,0,Attchd,1983.0,2,2,504,3,4,2,240,0,0,0,0,0,0,MnPrv,NA,0,12,2007,WD,Normal,144000,1.0,3112
43,44,A,RL,9200,Pave,NA,IR1,Lvl,AllPub,CulDSac,Gtl,CollgCr,Norm,Norm,1Fam,1Story,5,6,1975,1980,Hip,CompShg,VinylSd,VinylSd,None,0.0,3,3,CBlock,4,3,3,2,280,4,491,167,938,GasA,3,...,938,0,0,938,1,0,1,0,3,1,3,5,8,0,0,Detchd,1977.0,1,1,308,3,3,2,145,0,0,0,0,0,0,MnPrv,NA,0,7,2008,WD,Normal,130250,1.0,3122
50,51,F,RL,13869,Pave,NA,IR2,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1997,1997,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,3,PConc,4,3,3,6,182,1,0,612,794,GasA,4,...,794,676,0,1470,0,1,2,0,3,1,3,6,8,0,0,Attchd,1997.0,3,2,388,3,3,2,0,75,0,0,0,0,0,NA,NA,0,7,2007,WD,Normal,177000,2.0,4122
64,65,F,RL,9375,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,573.0,3,3,PConc,4,3,1,6,739,1,0,318,1057,GasA,5,...,1057,977,0,2034,1,0,2,1,3,1,4,8,8,0,0,Attchd,1998.0,2,2,645,3,3,2,576,36,0,0,0,0,0,GdPrv,NA,0,2,2009,WD,Normal,219500,2.5,5770


In [94]:
train_notnan.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,All_Bath,Total_Area
0,1,F,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4,3,PConc,4,3,1,6,706,1,0,150,856,GasA,...,856,854,0,1710,1,0,2,1,3,1,4,8,8,0,0,Attchd,2003.0,2,2,548,3,3,2,0,61,0,0,0,0,0,NA,NA,0,2,2008,WD,Normal,208500,2.5,4824
1,2,A,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,CBlock,4,3,4,5,978,1,0,284,1262,GasA,...,1262,0,0,1262,0,1,2,0,3,1,3,6,8,1,3,Attchd,1976.0,2,2,460,3,3,2,298,0,0,0,0,0,0,NA,NA,0,5,2007,WD,Normal,181500,2.0,4246
2,3,F,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,4,3,PConc,4,3,2,6,486,1,0,434,920,GasA,...,920,866,0,1786,1,0,2,1,3,1,4,6,8,1,3,Attchd,2001.0,2,2,608,3,3,2,0,42,0,0,0,0,0,NA,NA,0,9,2008,WD,Normal,223500,2.5,5100
3,4,G,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,3,3,BrkTil,3,4,1,5,216,1,0,540,756,GasA,...,961,756,0,1717,1,0,1,0,3,1,4,7,8,1,4,Detchd,1998.0,1,3,642,3,3,2,0,35,272,0,0,0,0,NA,NA,0,2,2006,WD,Abnorml,140000,1.0,4832
4,5,F,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,4,3,PConc,4,3,3,6,655,1,0,490,1145,GasA,...,1145,1053,0,2198,1,0,2,1,4,1,4,9,8,1,3,Attchd,2000.0,2,3,836,3,3,2,192,84,0,0,0,0,0,NA,NA,0,12,2008,WD,Normal,250000,2.5,6377
5,6,E,RL,85.0,14115,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,3,Wood,4,3,1,6,732,1,0,64,796,GasA,...,796,566,0,1362,1,0,1,1,1,1,3,5,8,0,0,Attchd,1993.0,1,2,480,3,3,2,40,30,0,320,0,0,0,MnPrv,Shed,700,10,2009,WD,Normal,143000,1.5,4000
6,7,A,RL,75.0,10084,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,4,3,PConc,5,3,3,6,1369,1,0,317,1686,GasA,...,1694,0,0,1694,1,0,2,0,3,1,4,7,8,1,4,Attchd,2004.0,2,2,636,3,3,2,255,57,0,0,0,0,0,NA,NA,0,8,2007,WD,Normal,307000,2.0,5710
8,9,E,RM,51.0,6120,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,3,3,BrkTil,3,3,1,1,0,1,0,952,952,GasA,...,1022,752,0,1774,0,0,2,0,2,2,3,8,7,2,3,Detchd,1931.0,1,2,468,2,3,2,90,0,205,0,0,0,0,NA,NA,0,4,2008,WD,Abnorml,129900,2.0,4968
9,10,Q,RL,50.0,7420,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,BrkTil,3,3,1,6,851,1,0,140,991,GasA,...,1077,0,0,1077,1,0,1,0,2,2,3,5,8,2,3,Attchd,1939.0,2,1,205,4,3,2,0,4,0,0,0,0,0,NA,NA,0,1,2008,WD,Normal,118000,1.0,3350
10,11,A,RL,70.0,11200,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1965,1965,Hip,CompShg,HdBoard,HdBoard,None,0.0,3,3,CBlock,3,3,1,3,906,1,0,134,1040,GasA,...,1040,0,0,1040,1,0,1,0,3,1,3,5,8,0,0,Detchd,1965.0,1,1,384,3,3,2,0,0,0,0,0,0,0,NA,NA,0,2,2008,WD,Normal,129500,1.0,3504


# Prediction of NaN LotFrontage with RandomForest

In [95]:
#Append both (numerical categorical, float and integer values) in one list of features to train in to RandomForestRegressor
feat_RF_lot = ['LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
'LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','GarageArea',
'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal', 'OverallQual', 'OverallCond','ExterQual', 'ExterCond',
'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','HeatingQC','CentralAir', 'KitchenQual', 'Functional', 'Fireplaces','FireplaceQu',
'GarageFinish','GarageCars', 'GarageQual', 'GarageCond','PavedDrive', 'YrSold','All_Bath']


In [96]:
X = train_notnan[feat_RF_lot]
y = train_notnan['LotFrontage']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.4)

In [97]:
forest = RandomForestRegressor(max_depth=9,n_estimators=150,max_leaf_nodes=None, min_samples_leaf=7, min_samples_split=2,random_state=0).fit(X_train,y_train)

In [98]:
#Analyzing the features that most influenced the model

importance_lot = pd.DataFrame({'Features': X.columns,'Importance':forest.feature_importances_}).sort_values(by='Importance',ascending=False)

features_importance_lot = list(importance_lot.Features)


In [99]:
importance_lot

,Features,Importance
0,LotArea,0.625507
1,YearBuilt,0.055704
19,GarageArea,0.037798
8,1stFlrSF,0.030672
11,GrLivArea,0.027106
42,GarageFinish,0.020391
20,WoodDeckSF,0.020144
6,BsmtUnfSF,0.019548
7,TotalBsmtSF,0.016895
2,YearRemodAdd,0.016430


In [100]:
#There are 42 features influenced
importance_lot[importance_lot.Importance!=0].count()

Features      42
Importance    42
dtype: int64

In [101]:
f3, ax3 = plt.subplots(figsize=(20,9))
sns.barplot(y='Features',x='Importance',data=importance_lot[:20],orient='h')

# 3.10.9.1. RandomForest with just features importance > 0




In [102]:
X = X[features_importance_lot[:42]]
y = train_notnan['LotFrontage']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.4)

In [103]:
'''param = {'n_estimators':[200],
         'max_depth':[None,1],
         'min_samples_split':[None,2,3],
         'min_samples_leaf':[9,10,11],
         'max_leaf_nodes':[None,1]     
         }
forest_cv = GridSearchCV(RandomForestRegressor(n_jobs=-1,random_state=0),param,cv=5,n_jobs=-1, scoring='neg_root_mean_squared_error').fit(X_train,y_train)

print('Melhores parâmetros RandomForest: {}'.format(forest_cv.best_params_))
print('Melhor score RandomForest: {}'.format(forest_cv.best_score_))'''

"param = {'n_estimators':[200],\n         'max_depth':[None,1],\n         'min_samples_split':[None,2,3],\n         'min_samples_leaf':[9,10,11],\n         'max_leaf_nodes':[None,1]     \n         }\nforest_cv = GridSearchCV(RandomForestRegressor(n_jobs=-1,random_state=0),param,cv=5,n_jobs=-1, scoring='neg_root_mean_squared_error').fit(X_train,y_train)\n\nprint('Melhores parâmetros RandomForest: {}'.format(forest_cv.best_params_))\nprint('Melhor score RandomForest: {}'.format(forest_cv.best_score_))"

In [104]:
forest = RandomForestRegressor(n_estimators=200,max_depth=None,max_leaf_nodes=None,min_samples_leaf=10,min_samples_split=2,random_state=0,n_jobs=1).fit(X_train,y_train)
#forest_cross = cross_val_score(forest,X_train,y_train,scoring='neg_root_mean_squared_error',n_jobs=-1,cv=10)
forest_cross = cross_val_score(forest,X_train,y_train,n_jobs=-1,cv=10, scoring='neg_root_mean_squared_error')


In [105]:
print('Cross validation Scores:\n', (forest_cross))
print('Cross validation Scores Média:\n', (forest_cross.mean()))
print('Score test {:.4f}'.format(forest.score(X_test,y_test)))
print('Score train: {:.4f}'.format(forest.score(X_train,y_train)))

Cross validation Scores:
 [-13.16520681 -16.9012004  -11.299921   -12.67997819 -16.50999285
 -18.83842046 -21.19235689 -17.55867007 -16.02432993 -15.9020713 ]
Cross validation Scores Média:
 -16.00721478992692
Score test 0.5511
Score train: 0.6701


In [106]:
y_pred_lot= forest.predict(train_nan[features_importance_lot[:42]])
y_pred_lot = y_pred_lot.round()

In [107]:
y_pred_lot_true= forest.predict(test_nan_true[features_importance_lot[:42]])
y_pred_lot_true = y_pred_lot_true.round()

In [108]:
lot_predict = pd.DataFrame({'Id':train_nan.index,'LotFrontage':y_pred_lot})
lot_predict = lot_predict.set_index('Id')
lot_predict_ = lot_predict.round()


In [109]:
lot_predict_true = pd.DataFrame({'Id':test_nan_true.index,'LotFrontage':y_pred_lot_true})
lot_predict_true = lot_predict_true.set_index('Id')
lot_predict_true_ = lot_predict_true.round()

# 3.10.9.2. Null values define with RandomForest


In [110]:
train7 = train6.copy()
test7 = test6.copy()

In [111]:
drop3 = train7.fillna({'LotFrontage': lot_predict_.LotFrontage})

In [112]:
train7.LotFrontage.head(10)

0    65.0
1    80.0
2    68.0
3    60.0
4    84.0
5    85.0
6    75.0
7     NaN
8    51.0
9    50.0
Name: LotFrontage, dtype: float64

In [113]:
drop3.LotFrontage.head(10)

0    65.0
1    80.0
2    68.0
3    60.0
4    84.0
5    85.0
6    75.0
7    85.0
8    51.0
9    50.0
Name: LotFrontage, dtype: float64

In [114]:
drop3_true = test7.fillna({'LotFrontage':lot_predict_true_.LotFrontage})

In [115]:
drop3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1457 entries, 0 to 1459
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1457 non-null   int64   
 1   MSSubClass     1457 non-null   category
 2   MSZoning       1457 non-null   category
 3   LotFrontage    1457 non-null   float64 
 4   LotArea        1457 non-null   int64   
 5   Street         1457 non-null   category
 6   Alley          1457 non-null   category
 7   LotShape       1457 non-null   category
 8   LandContour    1457 non-null   category
 9   Utilities      1457 non-null   category
 10  LotConfig      1457 non-null   category
 11  LandSlope      1457 non-null   category
 12  Neighborhood   1457 non-null   category
 13  Condition1     1457 non-null   category
 14  Condition2     1457 non-null   category
 15  BldgType       1457 non-null   category
 16  HouseStyle     1457 non-null   category
 17  OverallQual    1457 non-null   ca

In [116]:
drop3_true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1459 non-null   int64   
 1   MSSubClass     1459 non-null   category
 2   MSZoning       1459 non-null   category
 3   LotFrontage    1459 non-null   float64 
 4   LotArea        1459 non-null   int64   
 5   Street         1459 non-null   category
 6   Alley          1459 non-null   category
 7   LotShape       1459 non-null   category
 8   LandContour    1459 non-null   category
 9   Utilities      1459 non-null   category
 10  LotConfig      1459 non-null   category
 11  LandSlope      1459 non-null   category
 12  Neighborhood   1459 non-null   category
 13  Condition1     1459 non-null   category
 14  Condition2     1459 non-null   category
 15  BldgType       1459 non-null   category
 16  HouseStyle     1459 non-null   category
 17  OverallQual    1459 non-null   ca

# 5.  Data analysis of SalePrice (target)

In [117]:
train8 = drop3.copy()
test8 = drop3_true.copy()

In [118]:
train8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1457 entries, 0 to 1459
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1457 non-null   int64   
 1   MSSubClass     1457 non-null   category
 2   MSZoning       1457 non-null   category
 3   LotFrontage    1457 non-null   float64 
 4   LotArea        1457 non-null   int64   
 5   Street         1457 non-null   category
 6   Alley          1457 non-null   category
 7   LotShape       1457 non-null   category
 8   LandContour    1457 non-null   category
 9   Utilities      1457 non-null   category
 10  LotConfig      1457 non-null   category
 11  LandSlope      1457 non-null   category
 12  Neighborhood   1457 non-null   category
 13  Condition1     1457 non-null   category
 14  Condition2     1457 non-null   category
 15  BldgType       1457 non-null   category
 16  HouseStyle     1457 non-null   category
 17  OverallQual    1457 non-null   ca

In [119]:
corr_= train8.corr().drop('Id', axis=1).drop('Id', axis=0)
corr_price_ = corr_.SalePrice.sort_values(ascending=False)
corr_price_pos = corr_price_[corr_price_>=0].sort_values(ascending=False)
corr_price_neg = corr_price_[corr_price_<0].sort_values()

corr_price = corr_price_pos.append(corr_price_neg)
features_cont_price = list(corr_price.index.drop('SalePrice'))
corr_price_

SalePrice        1.000000
Total_Area       0.831273
GrLivArea        0.725475
TotalBsmtSF      0.645216
GarageArea       0.629498
1stFlrSF         0.624301
All_Bath         0.599035
FullBath         0.560701
TotRmsAbvGrd     0.535887
YearBuilt        0.525154
YearRemodAdd     0.510090
MasVnrArea       0.478164
BsmtFinSF1       0.405391
LotFrontage      0.374718
WoodDeckSF       0.327216
2ndFlrSF         0.322417
OpenPorchSF      0.320416
HalfBath         0.286920
LotArea          0.277007
BsmtFullBath     0.224018
BsmtUnfSF        0.217426
BedroomAbvGr     0.167730
ScreenPorch      0.112258
PoolArea         0.099853
3SsnPorch        0.044922
BsmtHalfBath    -0.016394
BsmtFinSF2      -0.019512
MiscVal         -0.021060
LowQualFinSF    -0.025421
EnclosedPorch   -0.128159
KitchenAbvGr    -0.135795
Name: SalePrice, dtype: float64

In [120]:
features_cont_price

['Total_Area',
 'GrLivArea',
 'TotalBsmtSF',
 'GarageArea',
 '1stFlrSF',
 'All_Bath',
 'FullBath',
 'TotRmsAbvGrd',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'LotFrontage',
 'WoodDeckSF',
 '2ndFlrSF',
 'OpenPorchSF',
 'HalfBath',
 'LotArea',
 'BsmtFullBath',
 'BsmtUnfSF',
 'BedroomAbvGr',
 'ScreenPorch',
 'PoolArea',
 '3SsnPorch',
 'KitchenAbvGr',
 'EnclosedPorch',
 'LowQualFinSF',
 'MiscVal',
 'BsmtFinSF2',
 'BsmtHalfBath']

In [121]:
f2, ax2 = plt.subplots(figsize=(20,9))
sns.heatmap(corr_,annot=True,center=1)

# 6. Machine Learning to predict the SalePrice values

# 6.2. Data manipulation for ML Linear Regression
FAZER ANÁLISE DE OUTLIERS DO SALEPRICE

In [122]:
encode2 = ['MSSubClass','MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1',
'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','HeatingQC',
 'CentralAir','Electrical','GarageType','Fence','MiscFeature','SaleType','SaleCondition']


In [123]:
test9 = test8.copy()
train9 = train8.copy()

In [124]:
train_reg = pd.get_dummies(train9,columns=encode2, drop_first=True)
test_reg = pd.get_dummies(test9,columns=encode2, drop_first=True)

In [125]:
train_reg.shape

(1457, 228)

In [126]:
test_reg.shape

(1459, 212)

In [127]:
encode_test = [ele for ele in train_reg.columns if ele in test_reg.columns] 
len(encode_test)

211

In [128]:
encode_test

['Id',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'MiscVal',
 'MoSold',
 'YrSold',
 'All_Bath',
 'Total_Area',
 'MSSubClass_B',
 'MSSubClass_C',
 'MSSubClass_D',
 'MSSubClass_E',
 'MSSubClass_F',
 'MSSubClass_G',
 'MSSubClass_H',
 'MSSubClass_I',
 'MSSubClass_J',
 'MSSubClass_L',
 'MSSubClass_M',
 'MSSubClass_O',
 'MSSubClass_P'

In [129]:
encode_train = ['Id','SalePrice',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'MiscVal',
 'MoSold',
 'YrSold',
 'All_Bath',
 'Total_Area',
 'MSSubClass_B',
 'MSSubClass_C',
 'MSSubClass_D',
 'MSSubClass_E',
 'MSSubClass_F',
 'MSSubClass_G',
 'MSSubClass_H',
 'MSSubClass_I',
 'MSSubClass_J',
 'MSSubClass_L',
 'MSSubClass_M',
 'MSSubClass_O',
 'MSSubClass_P',
 'MSSubClass_Q',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Pave',
 'Alley_NA',
 'Alley_Pave',
 'LotShape_IR2',
 'LotShape_IR3',
 'LotShape_Reg',
 'LandContour_HLS',
 'LandContour_Low',
 'LandContour_Lvl',
 'LotConfig_CulDSac',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'LotConfig_Inside',
 'LandSlope_Mod',
 'LandSlope_Sev',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_Gilbert',
 'Neighborhood_IDOTRR',
 'Neighborhood_MeadowV',
 'Neighborhood_Mitchel',
 'Neighborhood_NAmes',
 'Neighborhood_NPkVill',
 'Neighborhood_NWAmes',
 'Neighborhood_NoRidge',
 'Neighborhood_NridgHt',
 'Neighborhood_OldTown',
 'Neighborhood_SWISU',
 'Neighborhood_Sawyer',
 'Neighborhood_SawyerW',
 'Neighborhood_Somerst',
 'Neighborhood_StoneBr',
 'Neighborhood_Timber',
 'Neighborhood_Veenker',
 'Condition1_Feedr',
 'Condition1_Norm',
 'Condition1_PosA',
 'Condition1_PosN',
 'Condition1_RRAe',
 'Condition1_RRAn',
 'Condition1_RRNe',
 'Condition1_RRNn',
 'Condition2_Feedr',
 'Condition2_Norm',
 'Condition2_PosA',
 'Condition2_PosN',
 'BldgType_2fmCon',
 'BldgType_Duplex',
 'BldgType_Twnhs',
 'BldgType_TwnhsE',
 'HouseStyle_1.5Unf',
 'HouseStyle_1Story',
 'HouseStyle_2.5Unf',
 'HouseStyle_2Story',
 'HouseStyle_SFoyer',
 'HouseStyle_SLvl',
 'RoofStyle_Gable',
 'RoofStyle_Gambrel',
 'RoofStyle_Hip',
 'RoofStyle_Mansard',
 'RoofStyle_Shed',
 'RoofMatl_Tar&Grv',
 'RoofMatl_WdShake',
 'RoofMatl_WdShngl',
 'Exterior1st_AsphShn',
 'Exterior1st_BrkComm',
 'Exterior1st_BrkFace',
 'Exterior1st_CBlock',
 'Exterior1st_CemntBd',
 'Exterior1st_HdBoard',
 'Exterior1st_MetalSd',
 'Exterior1st_Plywood',
 'Exterior1st_Stucco',
 'Exterior1st_VinylSd',
 'Exterior1st_Wd Sdng',
 'Exterior1st_WdShing',
 'Exterior2nd_AsphShn',
 'Exterior2nd_Brk Cmn',
 'Exterior2nd_BrkFace',
 'Exterior2nd_CBlock',
 'Exterior2nd_CmentBd',
 'Exterior2nd_HdBoard',
 'Exterior2nd_ImStucc',
 'Exterior2nd_MetalSd',
 'Exterior2nd_Plywood',
 'Exterior2nd_Stone',
 'Exterior2nd_Stucco',
 'Exterior2nd_VinylSd',
 'Exterior2nd_Wd Sdng',
 'Exterior2nd_Wd Shng',
 'MasVnrType_BrkFace',
 'MasVnrType_None',
 'MasVnrType_Stone',
 'Foundation_CBlock',
 'Foundation_PConc',
 'Foundation_Slab',
 'Foundation_Stone',
 'Foundation_Wood',
 'Heating_GasW',
 'Heating_Grav',
 'Heating_Wall',
 'HeatingQC_2',
 'HeatingQC_4',
 'HeatingQC_1',
 'HeatingQC_3',
 'CentralAir_1',
 'Electrical_FuseF',
 'Electrical_FuseP',
 'Electrical_SBrkr',
 'GarageType_Attchd',
 'GarageType_Basment',
 'GarageType_BuiltIn',
 'GarageType_CarPort',
 'GarageType_Detchd',
 'GarageType_NA',
 'Fence_GdWo',
 'Fence_MnPrv',
 'Fence_MnWw',
 'Fence_NA',
 'MiscFeature_NA',
 'MiscFeature_Othr',
 'MiscFeature_Shed',
 'SaleType_CWD',
 'SaleType_Con',
 'SaleType_ConLD',
 'SaleType_ConLI',
 'SaleType_ConLw',
 'SaleType_New',
 'SaleType_Oth',
 'SaleType_WD',
 'SaleCondition_AdjLand',
 'SaleCondition_Alloca',
 'SaleCondition_Family',
 'SaleCondition_Normal',
 'SaleCondition_Partial']

In [130]:
train_reg[encode_train].shape

(1457, 212)

In [131]:
test_reg[encode_test].shape

(1459, 211)

In [132]:
corr2_= train_reg[encode_train].corr().SalePrice.sort_values(ascending= False).drop(['Id'])
corr2_neg = corr2_[corr2_<0].sort_values()
corr2 = corr2_[corr2_>=0]

corr_all= corr2.append(corr2_neg)
features = list(corr_all.index)[1:]
features

['Total_Area',
 'GrLivArea',
 'TotalBsmtSF',
 'GarageArea',
 '1stFlrSF',
 'All_Bath',
 'FullBath',
 'TotRmsAbvGrd',
 'YearBuilt',
 'YearRemodAdd',
 'Foundation_PConc',
 'MasVnrArea',
 'BsmtFinSF1',
 'Neighborhood_NridgHt',
 'MSSubClass_F',
 'LotFrontage',
 'SaleType_New',
 'SaleCondition_Partial',
 'GarageType_Attchd',
 'MasVnrType_Stone',
 'Neighborhood_NoRidge',
 'WoodDeckSF',
 '2ndFlrSF',
 'OpenPorchSF',
 'Exterior2nd_VinylSd',
 'Exterior1st_VinylSd',
 'HalfBath',
 'LotArea',
 'CentralAir_1',
 'HouseStyle_2Story',
 'MSZoning_RL',
 'Electrical_SBrkr',
 'GarageType_BuiltIn',
 'RoofStyle_Hip',
 'BsmtFullBath',
 'BsmtUnfSF',
 'Neighborhood_StoneBr',
 'MasVnrType_BrkFace',
 'Fence_NA',
 'RoofMatl_WdShngl',
 'BedroomAbvGr',
 'LotConfig_CulDSac',
 'Neighborhood_Somerst',
 'Exterior1st_CemntBd',
 'Exterior2nd_CmentBd',
 'LotShape_IR2',
 'Alley_NA',
 'LandContour_HLS',
 'Neighborhood_Timber',
 'Condition1_Norm',
 'ScreenPorch',
 'PoolArea',
 'MSZoning_FV',
 'Exterior2nd_ImStucc',
 'Neighborh

In [ ]:
y = train_reg['SalePrice']
def train_model(features):
  if len(features) == 0:
    return None
  model = LinearRegression().fit(train_reg[features], y)
  return model

def score_model(model, features):
  if len(features) == 0: 
    return dmba.AIC_score(y, [y.mean()] * len(y), model, df=1)
  return dmba.AIC_score(y, model.predict(train_reg[features]), model)

best_model, best_variables = dmba.stepwise_selection(features, train_model, score_model, verbose=True)
print(f'Intercept: {best_model.intercept_:.3f}')
print('Coefficients:') 
for name, coef in zip(best_variables, best_model.coef_):
  print(f' {name}: {coef}')

Variables: Total_Area, GrLivArea, TotalBsmtSF, GarageArea, 1stFlrSF, All_Bath, FullBath, TotRmsAbvGrd, YearBuilt, YearRemodAdd, Foundation_PConc, MasVnrArea, BsmtFinSF1, Neighborhood_NridgHt, MSSubClass_F, LotFrontage, SaleType_New, SaleCondition_Partial, GarageType_Attchd, MasVnrType_Stone, Neighborhood_NoRidge, WoodDeckSF, 2ndFlrSF, OpenPorchSF, Exterior2nd_VinylSd, Exterior1st_VinylSd, HalfBath, LotArea, CentralAir_1, HouseStyle_2Story, MSZoning_RL, Electrical_SBrkr, GarageType_BuiltIn, RoofStyle_Hip, BsmtFullBath, BsmtUnfSF, Neighborhood_StoneBr, MasVnrType_BrkFace, Fence_NA, RoofMatl_WdShngl, BedroomAbvGr, LotConfig_CulDSac, Neighborhood_Somerst, Exterior1st_CemntBd, Exterior2nd_CmentBd, LotShape_IR2, Alley_NA, LandContour_HLS, Neighborhood_Timber, Condition1_Norm, ScreenPorch, PoolArea, MSZoning_FV, Exterior2nd_ImStucc, Neighborhood_CollgCr, MiscFeature_NA, Neighborhood_Crawfor, Neighborhood_Veenker, MSSubClass_M, Neighborhood_ClearCr, Condition1_PosN, Condition2_PosN, Condition2

# 6.3. Training the models

# 6.3.1. SearchGrid
- **SGD**: 
- **LinearRegression**
- **Lasso**
- **ElasticNet**
- **RandomForestRegressor**
- **SVR**

# 6.3.1.1. Ridge
Best params Ridge: {'alpha': 30, 'solver': 'lsqr'}
Best score Ridge: 0.13842013339853912

In [ ]:
  y = np.log(train_reg['SalePrice'])
  X = StandardScaler().fit_transform(train_reg[best_variables])
  #X = train_reg[features_price_reg[:37]]
  X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3)

In [ ]:
#scoring='neg_mean_squared_log_error'
rg = Ridge(random_state=0)
param = {'alpha':range(1,31,1),
         'solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}

rg_cv = GridSearchCV(rg,param,cv=20, scoring='neg_root_mean_squared_error').fit(X_train,y_train)


print('Best params Ridge: {}'.format(rg_cv.best_params_))
print('Best score Ridge: {}'.format(-rg_cv.best_score_))

Best params Ridge: {'alpha': 30, 'solver': 'lsqr'}
Best score Ridge: 0.13842013339853912


10.460242108190519

In [ ]:
rg = Ridge(alpha=30,solver='lsqr',random_state=0).fit(X_train,y_train)
rg_cv = cross_val_score(rg,X_train,y_train,n_jobs=-1,cv=20,scoring='neg_root_mean_squared_error')

RMSE = (-rg_cv) 
print('Cross validation Scores:\n', (RMSE))
print('Cross validation Scores Média:\n', (RMSE.mean()))

y_pred_rd = rg.predict(X_test)

RMSE_pred = math.sqrt(mean_squared_error(y_test,y_pred_rd))

print('RMSE', RMSE_pred)
#print('Score test {:.4f}'.format(rg.score(X_test,y_test)))
#print('Score train: {:.4f}'.format(rg.score(X_train,y_train)))

Cross validation Scores:
 [0.10513085 0.14236654 0.1205279  0.18436187 0.1074986  0.16314666
 0.27622777 0.15253701 0.15936637 0.15223114 0.11156554 0.14560245
 0.14060339 0.09353307 0.11990439 0.10624357 0.10518971 0.12530378
 0.11549861 0.14156344]
Cross validation Scores Média:
 0.13842013339854278
RMSE 0.15294789270616352


# 6.3.1.2. Lasso
Best params Lasso: {'alpha': 0.00072}
Best score Lasso: 0.138878486480926

In [ ]:
  y = np.log(train_reg['SalePrice'])
  X = StandardScaler().fit_transform(train_reg[best_variables])
  #X = train_reg[features_price_reg[:37]]
  X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3)

In [ ]:
np.linspace(0.001,0.0005,21)

array([0.001   , 0.000975, 0.00095 , 0.000925, 0.0009  , 0.000875,
       0.00085 , 0.000825, 0.0008  , 0.000775, 0.00075 , 0.000725,
       0.0007  , 0.000675, 0.00065 , 0.000625, 0.0006  , 0.000575,
       0.00055 , 0.000525, 0.0005  ])

In [ ]:
#scoring='neg_mean_squared_log_error'
ls = Lasso(random_state=0)
param = { 'alpha': np.linspace(0.001,0.0005,51)}

ls_cv = GridSearchCV(ls,param,cv=20,scoring='neg_root_mean_squared_error').fit(X_train,y_train)

print('Best params Lasso: {}'.format(ls_cv.best_params_))
print('Best score Lasso: {}'.format((-ls_cv.best_score_)))

Best params Lasso: {'alpha': 0.0007199999999999999}
Best score Lasso: 0.138878486480926


In [ ]:
ls = Lasso(alpha=0.00072,random_state=0).fit(X_train,y_train)
ls_cv = cross_val_score(ls,X_train,y_train,n_jobs=-1,cv=20, scoring = 'neg_root_mean_squared_error' )

RMSE = (-ls_cv) 
print('Cross validation Scores:\n', (RMSE))
print('Cross validation Scores Média:\n', (RMSE.mean()))

y_pred_ls = ls.predict(X_test)

RMSLE_pred = math.sqrt(mean_squared_error(y_test,y_pred_ls))

print('RMSE', RMSLE_pred)

Cross validation Scores:
 [0.10768456 0.13716379 0.12052836 0.18320224 0.11087167 0.16457152
 0.28427565 0.15440278 0.16014543 0.1488585  0.1111417  0.14727956
 0.13786484 0.09261945 0.11877447 0.10456555 0.10498017 0.12412542
 0.11717888 0.14733518]
Cross validation Scores Média:
 0.138878486480926
RMSE 0.15409330959710493


# 6.3.1.3. ElasticNet
- Best params ElasticNet: {'alpha': 0.0058, 'selection': 'cyclic'}
Best score ElasticNet: 0.14350302187800174

In [ ]:
  y = np.log(train_reg['SalePrice'])
  X = StandardScaler().fit_transform(train_reg[best_variables])
  #X = train_reg[features_price_reg[:37]]
  X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3)

In [ ]:
np.linspace(0.001,0.01,91)

array([0.001 , 0.0011, 0.0012, 0.0013, 0.0014, 0.0015, 0.0016, 0.0017,
       0.0018, 0.0019, 0.002 , 0.0021, 0.0022, 0.0023, 0.0024, 0.0025,
       0.0026, 0.0027, 0.0028, 0.0029, 0.003 , 0.0031, 0.0032, 0.0033,
       0.0034, 0.0035, 0.0036, 0.0037, 0.0038, 0.0039, 0.004 , 0.0041,
       0.0042, 0.0043, 0.0044, 0.0045, 0.0046, 0.0047, 0.0048, 0.0049,
       0.005 , 0.0051, 0.0052, 0.0053, 0.0054, 0.0055, 0.0056, 0.0057,
       0.0058, 0.0059, 0.006 , 0.0061, 0.0062, 0.0063, 0.0064, 0.0065,
       0.0066, 0.0067, 0.0068, 0.0069, 0.007 , 0.0071, 0.0072, 0.0073,
       0.0074, 0.0075, 0.0076, 0.0077, 0.0078, 0.0079, 0.008 , 0.0081,
       0.0082, 0.0083, 0.0084, 0.0085, 0.0086, 0.0087, 0.0088, 0.0089,
       0.009 , 0.0091, 0.0092, 0.0093, 0.0094, 0.0095, 0.0096, 0.0097,
       0.0098, 0.0099, 0.01  ])

In [ ]:
en = ElasticNet(random_state=0)

param = { 'alpha':np.linspace(0.001,0.01,91),
         'selection': ['cyclic', 'random']}

en_cv = GridSearchCV(en,param,cv=20,scoring = 'neg_root_mean_squared_error').fit(X_train,y_train)

print('Best params ElasticNet: {}'.format(en_cv.best_params_))
print('Best score ElasticNet: {}'.format(-en_cv.best_score_))

Best params ElasticNet: {'alpha': 0.0015, 'selection': 'cyclic'}
Best score ElasticNet: 0.13885706877265946


In [ ]:
en = ElasticNet(alpha=0.0015, selection='cyclic').fit(X_train,y_train)
en_cv = cross_val_score(ls,X_train,y_train,n_jobs=-1,cv=20, scoring = 'neg_root_mean_squared_error' )

RMSE = (-en_cv) 
print('Cross validation Scores:\n', (RMSE))
print('Cross validation Scores Média:\n', (RMSE.mean()))

y_pred_en = en.predict(X_test)

RMSLE_pred = math.sqrt(mean_squared_error(y_test,y_pred_en))

print('RMSLE', RMSLE_pred)

Cross validation Scores:
 [0.10768456 0.13716379 0.12052836 0.18320224 0.11087167 0.16457152
 0.28427565 0.15440278 0.16014543 0.1488585  0.1111417  0.14727956
 0.13786484 0.09261945 0.11877447 0.10456555 0.10498017 0.12412542
 0.11717888 0.14733518]
Cross validation Scores Média:
 0.138878486480926
RMSLE 0.15398568511501712


Comparando com os valores do RMSLE

# 6.3.1.4. SGDRegressor
- BBest params SGDRegressor: {'alpha': 0.09100000000000001, 'learning_rate': 'invscaling', 'penalty': 'elasticnet'}
Best score SGDRegressor: 0.15541056071606102

In [ ]:
  y = np.log(train_reg['SalePrice'])
  X = StandardScaler().fit_transform(train_reg[best_variables])
  X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3)

In [ ]:
np.linspace(0.01,0.1,11)

array([0.01 , 0.019, 0.028, 0.037, 0.046, 0.055, 0.064, 0.073, 0.082,
       0.091, 0.1  ])

In [ ]:
#scoring= 'neg_mean_squared_log_error'

param = {'alpha':np.linspace(0.01,0.1,11),
           'penalty':['l2', 'l1', 'elasticnet'],
           #'epsilon':[],
           'learning_rate':['constant','optimal','invscaling','adaptive'],
           }

sgd_gr= GridSearchCV(SGDRegressor(random_state=0),param,n_jobs=-1,cv=20,scoring='neg_root_mean_squared_error').fit(X_train,y_train)

print('Best params SGDRegressor: {}'.format(sgd_gr.best_params_))
print('Best score SGDRegressor: {}'.format(-sgd_gr.best_score_))


Best params SGDRegressor: {'alpha': 0.09100000000000001, 'learning_rate': 'invscaling', 'penalty': 'elasticnet'}
Best score SGDRegressor: 0.15541056071606102


In [ ]:
sg = SGDRegressor(alpha=0.01,epsilon=1e-12, learning_rate='adaptive',penalty='elasticnet')
sg_cv = cross_val_score(ls,X_train,y_train,n_jobs=-1,cv=20,scoring='neg_root_mean_squared_error')

print('Cross validation Scores:\n', (-sg_cv))
print('Cross validation Scores Média:\n', (-sg_cv.mean()))

y_pred_sg = ls.predict(X_test)

RMSE_sg = math.sqrt(mean_squared_error(y_test,y_pred_sg))

print('RMSLE: predictions', RMSE_sg)
#print('Score test {:.4f}'.format(rg.score(X_test,y_test)))
#print('Score train: {:.4f}'.format(rg.score(X_train,y_train)))

Cross validation Scores:
 [0.10543949 0.15272407 0.17386989 0.26386354 0.13506428 0.08987874
 0.12366887 0.18442993 0.14374729 0.16524843 0.1598828  0.17604525
 0.10968582 0.11224697 0.17056347 0.12437832 0.11321484 0.13175646
 0.14336523 0.10644002]
Cross validation Scores Média:
 0.14427568626426276
RMSLE: predictions 0.20879142824867614


# 6.3.1.5. SVR --- AQUI
- 0.8619259783360235
SVR(C=1190, cache_size=200, coef0=0.0, degree=1, epsilon=34, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
  y = np.log(train_reg['SalePrice'])
  X = StandardScaler().fit_transform(train_reg[importance_price_all])
  X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3)

In [ ]:
#scoring= 'neg_mean_squared_log_error'

param = {'C':[0.1,1,5,10,15],
         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
         'degree':[1,2],
         'gamma': ['scale','auto'],
         'epsilon': [0.0001,0.001]#,0.01,0.1,1,2,5,10]
         }

svr_gr= GridSearchCV(SVR(),param,n_jobs=-1,cv=5,scoring= 'neg_root_mean_squared_error').fit(X_train,y_train)

print(-svr_gr.best_score_)
print(svr_gr.best_estimator_)

KeyboardInterrupt: ignored

In [ ]:
svr = SVR(C=1190,epsilon=34,gamma='scale',kernel='linear').fit(X_train,y_train)
svr_cv = cross_val_score(svr,X_train,y_train,n_jobs=-1,cv=20)

print('Cross validation Scores:\n', (svr_cv))
print('Cross validation Scores Média:\n', (svr_cv.mean()))

y_pred_svr = svr.predict(X_test)

MSLE_svr = math.sqrt(mean_squared_log_error(y_test,y_pred_svr))

print('RMSLE', MSLE_svr)
#print('Score test {:.4f}'.format(rg.score(X_test,y_test)))
#print('Score train: {:.4f}'.format(rg.score(X_train,y_train)))

Cross validation Scores:
 [0.90547404 0.92054564 0.83934902 0.85245242 0.7896909  0.83418686
 0.91373575 0.84776089 0.87804469 0.89982358 0.83381686 0.89540693
 0.88569624 0.84951516 0.83018575 0.77937148 0.81549417 0.93068668
 0.9029204  0.83436213]
Cross validation Scores Média:
 0.8619259783360235
RMSLE 0.26354374957909155


# 6.3.1.6. RandomForestRegressor
Best params for RandomForest: {'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 200}
Best score RandomForest: 0.1596322731937400

In [ ]:
  y = np.log(train_reg['SalePrice'])
  X = train_reg[importance_price_all]
  X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3)

In [ ]:
param = {'n_estimators':[200],
         'max_depth':[None,1,2,3,4],
         'min_samples_split':[None,1,2,3,4],
         'min_samples_leaf':[None,1,2,3],
         'max_leaf_nodes':[None,1,2,3]     
         }
forest_cv = GridSearchCV(RandomForestRegressor(n_jobs=-1,random_state=0),param,cv=5,n_jobs=-1,scoring= 'neg_root_mean_squared_error').fit(X_train,y_train)


print('Best params for RandomForest: {}'.format(forest_cv.best_params_))
print('Best score RandomForest: {}'.format(-forest_cv.best_score_))

Best params for RandomForest: {'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 200}
Best score RandomForest: 0.15963227319374007


# Analysing the most important features

In [ ]:
forest = RandomForestRegressor(max_depth=None,n_estimators=200,max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=3,random_state=0).fit(X_train,y_train)

In [ ]:
#Analyzing the features that most influenced the model

importance_price_RF = pd.DataFrame({'Features': X.columns,'Importance':forest.feature_importances_}).sort_values(by='Importance',ascending=False)

features_importance_price_RF = list(importance_price_RF.Features)


In [ ]:
importance_price_RF[:51]

,Features,Importance
5,YearBuilt,0.325913
0,GarageArea,0.136475
2,1stFlrSF,0.134652
1,TotalBsmtSF,0.054344
3,FullBath,0.047263
23,LotArea,0.043286
10,BsmtFinSF1,0.021499
4,TotRmsAbvGrd,0.019510
14,LotFrontage,0.018519
7,MasVnrArea,0.013690


- Best params for RandomForest: {'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 200}
Best score RandomForest: 0.1731887380962448

In [ ]:
#There are 203 features influenced
importance_price_RF[importance_price_RF.Importance!=0].count()

Features      203
Importance    203
dtype: int64

In [ ]:
X = train_reg[features_importance_price_RF[:203]]
y = np.log(train_reg['SalePrice'])
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3)

In [ ]:
#scoring='neg_mean_squared_log_error'
param = {'n_estimators':[200],
         'max_depth':[None,1,2],
         'min_samples_split':[2,3,4],
         'min_samples_leaf':[None,1,2,3],
         'max_leaf_nodes':[None,1,2,3]     
         }
forest_cv = GridSearchCV(RandomForestRegressor(n_jobs=-1,random_state=0),param,cv=5,n_jobs=-1,
                         scoring= 'neg_root_mean_squared_error').fit(X_train,y_train)
                         
print('Best params for RandomForest: {}'.format(forest_cv.best_params_))
print('Best score RandomForest: {}'.format(-forest_cv.best_score_))


Best params for RandomForest: {'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 200}
Best score RandomForest: 0.1731887380962448


# Kaggle ElasticNet
- 0.14427/0.205 
- Kaggle = 0.17047

In [ ]:
  y = np.log(train_reg['SalePrice'])
  X = StandardScaler().fit_transform(train_reg[importance_price_all])
  X_true = StandardScaler().fit_transform(test_reg[importance_price_all])

In [ ]:
en_k = ElasticNet(alpha=0.0058, selection='cyclic').fit(X,y)

y_pred_en_true1 = np.exp(en_k.predict(X_true))


In [ ]:
output = pd.DataFrame({'Id': test1.Id, 'SalePrice': y_pred_en_true1})
output.to_csv('Sale_Price[4]_EN1.csv', index=False)

In [ ]:
output

,Id,SalePrice
0,1461,1.312934e+05
1,1462,1.591282e+05
2,1463,1.901314e+05
3,1464,2.057717e+05
4,1465,2.175137e+05
5,1466,1.771969e+05
6,1467,1.940509e+05
7,1468,1.741964e+05
8,1469,1.734514e+05
9,1470,1.320264e+05
